In [ ]:
PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

!curl -L {DATA_URL} -o hair_classifier_v1.onnx.data
!curl -L {MODEL_URL} -o hair_classifier_v1.onnx

In [ ]:
!pip install onnx
!pip install onnxruntime

In [ ]:
from torchvision import transforms
from io import BytesIO
from urllib import request
from PIL import Image
import onnxruntime as ort
import numpy as np
import onnx

In [ ]:
# Load the model
model_path = "hair_classifier_v1.onnx"
model = onnx.load(model_path)

# Print the output node names
output_names = [output.name for output in model.graph.output]
print("Output node names:", output_names)


Output node names: ['output']


In [ ]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [46]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img = download_image(url)
img = prepare_image(img, target_size=(200, 200))

In [50]:
preprocess = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

# Apply the preprocessing
input_tensor = preprocess(img)

# Convert to numpy array (C x H x W)
input_array = input_tensor.numpy()

# Get the value of the first pixel in the R channel (index 0)
r_channel_first_pixel_value = input_array[0, 0, 0]  # [C, H, W] -> [0, 0, 0] gives the first pixel in R

print(f"Value of the first pixel in the R channel: {r_channel_first_pixel_value}")

Value of the first pixel in the R channel: -1.0732940435409546


In [ ]:
input_array = np.expand_dims(input_array, axis=0)
onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

result = session.run([output_name], {input_name: input_array})
predictions = result[0][0].tolist()
print("Predictions:", predictions)

Predictions: [0.09156641364097595]
